In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

MEDIA_DIR = os.path.abspath("")

In [ ]:
dbpath = os.path.join(MEDIA_DIR, "metadata_db.csv")
assert os.path.exists(dbpath)
df = pd.read_csv(dbpath).sort_values('Glicko_pts', ascending=False)
df

In [ ]:
extensions = df['name'].str.lower().str.rsplit('.', n=1, expand=True)[1].value_counts()
extensions.plot.bar(title="file extensions")

In [ ]:
df.plot(subplots=True, figsize=(10,10))

In [ ]:
toptags = df['tags'].str.split().explode().value_counts()
toptags[:60].plot.bar(figsize=(10,4), title="tag frequency")

In [ ]:
df.hist(column=['Glicko_pts', 'ELO_pts', 'stars'], bins=100, figsize=(20,6))